<a href="https://colab.research.google.com/github/BarryLiu-97/Pytorch-Tutorial/blob/master/07_Softmax_Classifier_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
batch_size = 64
transform = transforms.Compose([
  transforms.ToTensor(),              #ToTensor()将数据转换为channel×width×height格式，为了更高效地进行运算
  transforms.Normalize((0.1307, ), (0.3081, ))  #均值和标准差，用于数据标准化，这是对MNIST进行计算后得到的结果，已经算好了
])

train_dataset = datasets.MNIST(root='../dataset/mnist',
                train=True, download=True,
                transform = transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist',
                train=False, download=True,
                transform = transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [ ]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = torch.nn.Linear(784, 512)
    self.l2 = torch.nn.Linear(512, 256)
    self.l3 = torch.nn.Linear(256, 128)
    self.l4 = torch.nn.Linear(128, 64)
    self.l5 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 784)   
    #数据本身是(N×1×28×28)的Tensor。全来凝结神经网络要求输入是一个矩阵，而不是三维的张量。
    #view方法相当于reshape，28*28等于784，将每个图片的数据一行一行拼接，-1表示根据第二个人为设置的维度，自动计算第一个维度的大小
    x = F.elu(self.l1(x))
    x = F.elu(self.l2(x))
    x = F.elu(self.l3(x))
    x = F.elu(self.l4(x))
    return self.l5(x)  #最后一层不做激活

model = Net()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)  #冲量值设置为0.5，优化训练过程

In [ ]:
def train(epoch):
  running_loss = 0.0
  for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
    optimizer.zero_grad()
    # inputs = inputs.cuda()
    # target = target.cuda()
    #forward + backward + update
    outputs = model(inputs)
    loss = criterion(outputs, target) 
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 ==299:  
      #300次迭代输出一次loss
      # print('[%d, %5d] loss:%.3f' % (epoch + 1, batch_idx + 1, running_loss / 300)) 
      running_loss= 0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():#该缩进块中的代码不会计算梯度
    for data in test_loader:
      images, labels = data
      # images = images.cuda() 
      # labels = labels.cuda()
      outputs = model(images)
      _, predicted = torch.max(outputs.data, dim=1) #输出为列数为10，代表十个类别，取出每一行数值最大地那一列的下标，即获得该样本的类别，带下为(N×1)
      # predicted = predicted.cuda() 
      total += labels.size(0)   #取出的是N
      correct += (predicted == labels).sum().item()   #张量之间的比较运算
  print('Acc on test set:%d %%' % (100* correct / total))

In [ ]:
if __name__ == '__main__':
  start = time.time()
  for epoch in range(10):
    train(epoch)
    test()
  end = time.time()
  print(str(end-start) + 's')